# CUDA exercises

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import itertools
%matplotlib inline

from IPython.display import display

In [ ]:
def csv_parse(path, header=False, sep=";"):
    if header:
        df = pd.read_csv(path, sep=sep, names=header)
    else:
        df = pd.read_csv(path, sep=sep)
    #display(df.head())
    return df

In [ ]:
def calcGB_Stride(df):
    return 3*8* df["N"]/df["k"] /1e9

def calcGB_Offset(df):
    return 3*8* (df["N"]-df["k"])/1e9

def transform_to_ms(df, cols):
    return df[cols]*1e3


# Ex7.1

In [ ]:
def print_styling():

    plt.rc('figure', figsize=(14,8)) 
    plt.style.use("seaborn")
    sns.set_palette("Paired")
    SMALL_SIZE = 15
    MEDIUM_SIZE = 18
    BIGGER_SIZE = 26

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('lines', linewidth=2)

    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
print_styling()

In [ ]:
df_CPU = csv_parse("data/ph_data_CPU.csv", header=False)
display(df_CPU)
df_GPU = csv_parse("data/ph_data_GPU.csv", header=False)
display(df_GPU)
df_cuda = csv_parse("data/data_cuda.csv", header=False)
display(df_cuda)
df_cuda2 = csv_parse("data/data_cuda.csv", header=False)
display(df_cuda2)
data = [df_CPU, df_GPU, df_cuda2]

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(14,10))

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

ax = axs[0]
ax2 = axs[1]

print_styling();
for df in data:
    try:
        label = df["target"].unique()[0]
    except:
        label = "cuda"
    if label == "cuda":
        df.plot(x="N", y="time_per_test", ax=ax, logx=logx, logy=logy, label=label)
    else:
        df.plot(x="N", y="total_time", ax=ax, logx=logx, logy=logy, label=label)

        to_plot = ["ocl_time", "cpu_time"]
        to_label = ["kernel", "host reduction"]
        markers = ["x", "o"]
        ls = "--" if label=="GPU" else "-"
        labels = [f"{label}({x})" for x in to_label]
        df.plot(x="N", y=to_plot, ax=ax2, logx=logx, logy=logy, label=labels, linestyle=ls, marker="x")

ax.set_ylabel("runtime [s]")
ax.set_title("Total")
ax.set_ylim(ax2.get_ylim())
ax2.set_title("OCL time spent per part")
fig.suptitle("Dot product performances")
for a in axs.ravel():
    a.grid(True)

plt.tight_layout()
fig.savefig("plots/ex7_runtimes.pdf", bbox_inches="tight", pad_inches=0.2)

## Compilation

In [ ]:
df = csv_parse("data/ph_data_compile.csv", header=False)
df["create_time_M"] = df["create_time"]*df["M"]
display(df)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

print_styling();

df.plot(x="M", y=["compile_time", "create_time_M"], ax=ax, logx=logx, logy=logy, label=["compilation", "fetch M kernels"])

ax.set_ylabel("runtime [s]")
fig.suptitle("OpenCL: Compilation and kernel fetch times")
ax.grid(True)

plt.tight_layout()
fig.savefig("plots/ex7_compilation.pdf", bbox_inches="tight", pad_inches=0.2)